In [1]:
import pandas as pd
from pandas import Series
from pandas import DataFrame
import numpy as np

pandas基于numpy的数组结构创建的, 虽然保留了numpy的数组操作, 但其主要任务为处理表格和混杂数据

### 1.1 Series
#### 1. Series结构 = index + 一维数组.  
  1. index获取 : Series.index属性  
    1. 创建Series时, 若没有指定index, 则默认添加从0开始递增1的索引
    2. 若想创建Series时指定索引, 可以使用Series(values,index=[..])
  2. 一维数组获取 : Series.values属性  


In [2]:
obj = Series([1,3,2,5])
# Series显示 : 左侧为index,右侧为一维数组
print obj  
print "=============[1]============"
print obj.index
print obj.values
print "=============[2]============"
# 创建带index的Series
obj = Series([1,2,3,4],index=['a','b','c','d'])
obj.index

0    1
1    3
2    2
3    5
dtype: int64
=============[1]============
RangeIndex(start=0, stop=4, step=1)
[1 3 2 5]
=============[2]============


Index([u'a', u'b', u'c', u'd'], dtype='object')

#### 2. 与numpy不同, pandas可以通过索引选取元素  
  1. 使用Series[index]选取元素
  2. 使用Series[index] = newVal更新元素值
  3. 使用Series[[index1,index2..]]选取多个元素, 返回一个新的Series(index不变)
  3. 使用numpy的数组函数会保留Series的index. 仍返回Series

In [3]:
print "obj[a] = %s" % obj['a']
print "=============[1]============"
obj2 = obj[['a','b']]
obj2['b'] = 99
print obj2
print "========numpy的数组函数不改变Series的index======="
np.exp(obj2)

obj[a] = 1
=============[1]============
a     1
b    99
dtype: int64
========numpy的数组函数不改变Series的index=======


a    2.718282e+00
b    9.889030e+42
dtype: float64

#### 3. Series与字典
1. 可以看做一个定长有序字典, 因为他是多对(index->value)的映射. 因此Series可以使用大部分字典函数
2. 可以通过字典直接创建Series. 字典为(rowIdx->value)
3. 仅通过字典创建的Series, 其元素自动按照字典的key进行排列.若创建Series时指定index的顺序, 则Series元素按照index的顺序排序
4. 可以使用pandas的isnull判断Series的字典中的value是否为NaN数据 (空数据)

In [4]:
# 使用字典函数操作Series
print ('a' in obj)
# 通过字典直接创建Series
data = {'d':9,'b':5,'c':7,'a':1}
obj = Series(data)  # 自动排列
print data
print obj
print "=============[1]============"
obj = Series(data,index=['b','a','c','d'])
print obj
print "=============[2]============"
print pd.isnull(obj) # value是否为NaN

True
{'a': 1, 'c': 7, 'b': 5, 'd': 9}
a    1
b    5
c    7
d    9
dtype: int64
=============[1]============
b    5
a    1
c    7
d    9
dtype: int64
=============[2]============
b    False
a    False
c    False
d    False
dtype: bool


#### 4. Series的一些其他概念
1. Series之间进行数据运算时, 会按照index对齐数据
2. Series自身和其index对象, 都有name属性, 该属性和其他关键功能密切相关
3. Series的index可以通过赋值而改变

In [5]:
# index对齐数据
obj1 = Series([1,1,1,1],index=["a","b","c","d"])
obj2 = Series([1,2,3,4],index=["a","b","c","e"])
print obj1+obj2
print "=============[1]============"
# name属性
obj1.name = "population"
obj1.index.name = "state"
print obj1
print "=============[2]============"
# index改变
obj1.index = ['aa','bb','cc','dd']
print obj1

a    2.0
b    3.0
c    4.0
d    NaN
e    NaN
dtype: float64
=============[1]============
state
a    1
b    1
c    1
d    1
Name: population, dtype: int64
=============[2]============
aa    1
bb    1
cc    1
dd    1
Name: population, dtype: int64


## 1.2 DataFrame

#### 1. DataFrame基本概念
1. 一个DataFrame就是一个表格.由一组Series组成. 这些Series长度相同, 且共用一个索引
2. DataFrame也可看做Series组成的字典,字典形式为(columnIdx->Series)
3. DataFrame有航索引和列索引

#### 2. DataFrame创建
1. DataFrame也可以看做一个字典, 其字典形式为(columnIdx,一维数组).因此可以通过字典创建DataFrame, 
2. 若创建时同时指定了字典和columnIdx, 则会按照columnIdx的顺序构造DataFrame.   
 若columnIdx中包含字典中不存在的key, 则DataFrame这列值为NaN

In [6]:
data = {'name':['zhangsan','lisi','wangwu'],
       'age':[22,27,25]}
df = DataFrame(data)
print df
print ("=============[1]============")
df2 = DataFrame(data,columns=['age','name','score'])
print df2

   age      name
0   22  zhangsan
1   27      lisi
2   25    wangwu
=============[1]============
   age      name score
0   22  zhangsan   NaN
1   27      lisi   NaN
2   25    wangwu   NaN


#### 3. DataFrame元素选取
1. 通过DataFrame对应字典的key,选取一维数组. 返回Series
2. DataFrame对应字典的key已经作为DataFrame的属性存在. 因此可以通过DataFrame."key name"选取一维数组Series
[注]: 无论哪种方式选取DataFrame中的Series, 返回的Series都和DataFrame有相同的索引(但不共用)

In [7]:
print df['age']
print ("=============[1]============")
obj = df.age
print obj

0    22
1    27
2    25
Name: age, dtype: int64
=============[1]============
0    22
1    27
2    25
Name: age, dtype: int64


#### 4. 通过赋值方式更新DataFrame的Series
1. 指定DataFrame的某列Series为一个标量, 则该Series的所有元素值都等于这个标量值
2. 指定DataFrame的某列Series为一个新的Series, 则这两个Series之间的rowindex会一一对应进行匹配
3. del关键字可以物理删除DataFrame的一个Series
4. 通过DataFrame['newColumnIdx']添加一个新的Series

In [8]:
df2['score'] = 100
print df2
print ("==============[1]============")
df2['score'] = Series([99,100,98])
print df2
print ("==============[2]============")
del df2['score']
print df2
print ("==============[3]============")
df2['ageBool'] = df2['age']>25
print df2

   age      name  score
0   22  zhangsan    100
1   27      lisi    100
2   25    wangwu    100
==============[1]============
   age      name  score
0   22  zhangsan     99
1   27      lisi    100
2   25    wangwu     98
==============[2]============
   age      name
0   22  zhangsan
1   27      lisi
2   25    wangwu
==============[3]============
   age      name  ageBool
0   22  zhangsan    False
1   27      lisi     True
2   25    wangwu    False


#### 5. DataFrame的其他概念
1. DataFrame的创建方式有很多种, 例如
  1. DataFrame(二维ndArray)
  2. DataFrame(dict('columnIdx'->List))
  3. DataFrame(dict('columnIdx'->Series))
  4. DataFrame(dict('columnIdx'->dict('rowIdx'->标量值) ))
2. DataFrame的index和columns都由name属性

### 1.3 索引对象
#### 1. Index对象的基本概念
1. Index对象是不可变的, 因此可以再多个数据结构中共享Index对象. 使用pd.Index([数组])创建索引对象
2. Index对象类似固定大小的集合. 可使用关键字in查看集合中是否有某个元素
3. pandas的Index和python的集合不同, Index中可以包含重复数据

In [16]:
labels = pd.Index(range(0,3))
obj = Series([99,98,100],index=labels)
obj2 = DataFrame({'a':[1,2,3],'b':[4,4,4]},index=labels)
# is关键字: 判断id(obj1)和id(obj2)是否相等
# ==:判断2两对象的value是否相等
print obj.index is obj2.index
print 4 in labels
print ("==============[1]============")
# Index包含重复数据
labels = pd.Index(['a','a','b','c'])
obj1 = Series(range(0,4),index=labels)
print obj1

True
False
==============[1]============
a    0
a    1
b    2
c    3
dtype: int64


#### 2. 重建索引reindex : 使用新的索引来重构数据, 并可指定原先不包含的索引对应数据的填充策略 
1. Series的数据可以通过重建索引来重排数据 (返回新对象)
2. 指定原先不包含的索引对应数据的填充策略, 包括: ffill(front向前填充,直到填充到下一个有值的地方),bfill(backword向后填充,知道下一个有值得地方)  
3. DataFrame重建索引默认为行索引,重建列索引需要声明columns=[newIndex]

In [36]:
# 1. 重排数据
obj = Series([2,3,1,4],index=['b','c','a','d'])
obj2 = obj.reindex(['a','b','c','d','e'])
obj2
# 2. 填充缺失索引对应的数值
obj = Series(['a','b','c'],index=[2,4,6])
obj2 = obj.reindex(range(0,6),method='bfill')
print obj2
# 3. DataFrame重建索引
df = DataFrame(np.arange(0,9).reshape(3,3),index=['a','b','c'],columns=['apple','orange','banana'])
df2 = df.reindex(['c','b','d'])
print df2
df3 = df.reindex(columns=['apple','banana','xixi'])
df3

0    a
1    a
2    a
3    b
4    b
5    c
dtype: object
   apple  orange  banana
c    6.0     7.0     8.0
b    3.0     4.0     5.0
d    NaN     NaN     NaN


,apple,banana,xixi
a,0,2,NaN
b,3,5,NaN
c,6,8,NaN


#### 3. 删除指定索引上的一行或一列数据
1. Series和DataFrame都由drop方法, 使用时指定一个索引名/索引数组即可  
 DataFrame根据索引删除行/列时, axis=0为行, axis=1为列


In [44]:
obj = Series(np.arange(0,5),index=['a','b','c','d','e'])
obj2 = obj.drop('a')
print obj2
# DataFrame.drop(indexName) : axis=1为列,axis=0为行
df = DataFrame(np.arange(0,9).reshape(3,3),index=['a','b','c'],columns=['apple','orange','banana'])
df2 = df.drop(['apple','banana'],axis=1)
df2

b    1
c    2
d    3
e    4
dtype: int64


,orange
a,1
b,4
c,7


#### 4. 选取索引上的数据
1. 无论是Series还是DataFrame都能通过loc[],iloc[]来选取数据. loc按"索引值"选取, iloc按"索引序号"选取
2. 条件选取: 通常会根据一个where条件选取DataFrame中的某些行, pandas的实现逻辑分两种情况:  
  1. bool型Series作索引
    1. 将一个bool型Series,放到DataFrame的索引数组中进行选取. DataFrame只选取Series中value=True的索引
    2. 这些value=True的索引可供下一步进行索引选取
  2. bool型DataFrame做索引
    1. 将这个bool型DataFrame中为True元素的(rowIdx,columnIdx)作为组合去选取数据

In [82]:
df = DataFrame(np.arange(0,9).reshape(3,3),index=['a','b','c'],columns=['apple','orange','banana'])
print df
print ("==============[1]============")
# 下述选取方式等价
print df.loc[:,'apple']
print df.iloc[:,0]
print ("==============[2]============")
# 选取df中apple列大于1的所有行
print df.loc[df.apple>1,:]

   apple  orange  banana
a      0       1       2
b      3       4       5
c      6       7       8
==============[1]============
a    0
b    3
c    6
Name: apple, dtype: int64
a    0
b    3
c    6
Name: apple, dtype: int64
==============[2]============
   apple  orange  banana
b      3       4       5
c      6       7       8
==============[3]============


apple     True
orange    True
banana    True
Name: a, dtype: bool

#### 4. iloc和loc在索引值为整数情况下的小歧义
1. iloc由于是根据索引序号选取, 所以是[a,b)区间的选取  
 loc是索引值得选取, 所以为[a,b]区间的索引.  
 此时, 当索引值也为整数时, 就要格外小心, 选取的行/列是否包含最后一个值

In [89]:
df = DataFrame(np.arange(0,9).reshape(3,3))
print df.iloc[:1,0] # 只有rowIdx=0的行
print df.loc[:1,0] # 有rowIdx=0和1的两行

0    0
Name: 0, dtype: int64
0    0
1    3
Name: 0, dtype: int64


### 1.4 pandas的数据对齐
1. pandas的Series之间/DataFrame之间进行算术运算时, 需要对齐两个Series/DataFrame的索引再进行元素之间运算  
 如果存在没有对其的索引, 就会将值设置为缺失值NaN
2. 对于没对齐的索引, 所以通过设置fill_value属性, 填充双方没对齐的元素的缺省值(对df1,df2都成立)

In [98]:
df1 = DataFrame(np.arange(0,9).reshape(3,3),columns=list('abc'))
print df1
df2 = DataFrame(np.arange(0,9).reshape(3,3),columns=list('bcd'))
print df2
print "==============[1]============"
# 对df1的缺失索引元素与df2的缺失索引元素都填充0
print df1 + df2
print df1.add(df2,fill_value=0)

   a  b  c
0  0  1  2
1  3  4  5
2  6  7  8
   b  c  d
0  0  1  2
1  3  4  5
2  6  7  8
==============[1]============
    a   b   c   d
0 NaN   1   3 NaN
1 NaN   7   9 NaN
2 NaN  13  15 NaN
     a   b   c    d
0  0.0   1   3  2.0
1  3.0   7   9  5.0
2  6.0  13  15  8.0
